# Stock Chart Pattern Recognition with Deep Learning
## CRISP-DM Methodology

In [19]:
import pandas as pd
from cassandra.cluster import Cluster
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf

In [2]:
# 1. เชื่อมต่อ Cassandra (ตามโค้ดของคุณ)
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('stock_data')

In [3]:
session = cluster.connect('stock_data')
query = """
    SELECT symbol, date, open, high, low, close, volume
    FROM candlestick_data
        """
        
rows = session.execute(query)
df = pd.DataFrame(list(rows))
            
# จัดเรียงข้อมูลตามวันที่ (Time-series ordering)
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by=['symbol', 'date'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [4]:
df.head()

,symbol,date,open,high,low,close,volume
0,2S,2022-12-27,3.04,3.04,3.00,3.04,189255
1,2S,2022-12-28,3.04,3.04,3.00,3.02,87697
2,2S,2022-12-29,3.02,3.04,3.02,3.02,124965
3,2S,2022-12-30,3.00,3.08,3.00,3.04,109025
4,2S,2023-01-03,3.04,3.10,3.04,3.06,61485


In [5]:
print(df.describe())

                                date          open          high  \
count                          69711  69711.000000  69711.000000   
mean   2024-06-29 09:20:52.261479424     14.842662     14.987348   
min              2022-12-27 00:00:00      0.020000      0.020000   
25%              2023-09-28 00:00:00      1.400000      1.420000   
50%              2024-07-03 00:00:00      4.260000      4.320000   
75%              2025-03-27 00:00:00      9.650000      9.750000   
max              2025-12-26 00:00:00    318.000000    320.000000   
std                              NaN     36.914853     37.202065   

                low         close        volume  
count  69711.000000  69711.000000  6.971100e+04  
mean      14.677948     14.830940  6.509695e+06  
min        0.010000      0.010000  1.000000e+02  
25%        1.370000      1.390000  7.564450e+04  
50%        4.200000      4.260000  5.138000e+05  
75%        9.550000      9.650000  3.672491e+06  
max      315.000000    317.000000  1.

In [6]:
print(df.isnull().sum())

symbol    0
date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64


In [7]:
# ตรวจสอบว่าในแต่ละ Symbol มีข้อมูลกี่แถว
print(df.groupby('symbol').size())

# ดูช่วงวันที่ของข้อมูล
print(f"Start Date: {df['date'].min()}")
print(f"End Date: {df['date'].max()}")

symbol
2S        731
3BBIF     731
A         671
A5        727
AAI       731
         ... 
BE8       731
BEAUTY    731
BEC       731
BEM       731
BEYOND    728
Length: 99, dtype: int64
Start Date: 2022-12-27 00:00:00
End Date: 2025-12-26 00:00:00
